# fse - Tutorial

Welcome to fse - fast sentence embeddings. The library is intended to compute sentence embeddings as fast as possible. 
It offers a simple and easy to understand syntax for you to use in your own projects. Before we start with any model, lets have a look at the input types.
All fse models require an iterable/generator which produces a tuple. The tuple has two fields: words and index. The index is required for the multi-thread processing, as sentences might not be processed sequentially. The index dictates, which row of the corresponding sentence vector matrix the sentence belongs to.

## Input handling

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
s = (["Hello", "world"], 0)
print(s[0])
print(s[1])

['Hello', 'world']
0


The words of the tuple will always consist of a list of strings. Otherwise the train method will raise an Error. However, most input data is available as a list of strings.

In order to deal with this common input format, fse provides the IndexedList and some variants, which handel all required data operations for you. You can provide multiple lists (or sets) which will all be merged into a single list. This eases work if you have to work with the STS datasets.

The multiple types of indexed lists. Let's go through them one by one:
- IndexedList: for already pre-splitted sentences
- **C**IndexedList: for already pre-splitted sentences with a custom index for each sentence
- SplitIndexedList: for sentences which have not been splitted. Will split the strings
- Split**C**IndexedList: for sentences which have not been splitted and with a custom index for each sentence
- **C**SplitIndexedList: for sentences which have not been splitted. Will split the strings. You can provide a custom split function
- **C**Split*C*IndexedList: for sentences where you want to provide a custom index and a custom split function.

*Note*: These are ordered by speed. Meaning, that IndexedList is the fastest, while **C**Split*C*IndexedList is the slowest variant.

In [3]:
from fse import SplitIndexedList

sentences_a = ["Hello there", "how are you?"]
sentences_b = ["today is a good day", "Lorem ipsum"]

s = SplitIndexedList(sentences_a, sentences_b)
print(len(s))
s[0]

4


(['Hello', 'there'], 0)

To save memory, we do not convert the original lists inplace. The conversion will only take place once you call the getitem method. To access the original data, call:

In [4]:
s.items

['Hello there', 'how are you?', 'today is a good day', 'Lorem ipsum']

If the data is already preprocessed as a list of lists you can just use the IndexedList

In [5]:
from fse import IndexedList

sentences_splitted = ["Hello there".split(), "how are you?".split()]
s = IndexedList(sentences_splitted)
print(len(s))
s[0]

2


(['Hello', 'there'], 0)

In case you want to provide your own splitting function, you can pass a callable to the **C**SplitIndexedList class.

In [6]:
from fse import CSplitIndexedList

def split_func(string):
    return string.lower().split()

s = CSplitIndexedList(sentences_a, custom_split=split_func)
print(len(s))
s[0]

2


(['hello', 'there'], 0)

If you want to stream a file from disk (where each line corresponds to a sentence) you can use the IndexedLineDocument.

In [7]:
from fse import IndexedLineDocument
doc = IndexedLineDocument("../test/test_data/test_sentences.txt")

In [8]:
i = 0
for s in doc:
    print(f"{s[1]}\t{s[0]}")
    i += 1
    if i == 4:
        break

0	['Good', 'stuff', 'i', 'just', 'wish', 'it', 'lasted', 'longer']
1	['Hp', 'makes', 'qualilty', 'stuff']
2	['I', 'like', 'it']
3	['Try', 'it', 'you', 'will', 'like', 'it']


If you are later working with the similarity of sentences, the IndexedLineDocument provides you the option to access each line by its corresponding index. This helps you in determining the similarity of sentences, as the most_similar method would otherwise just return indices.

In [9]:
doc[20]

'I feel like i just got screwed'

# Training a model / Performing inference

Training a fse model is simple. You only need a pre-trained word embedding model which you use during the initializiation of the fse model you want to use.

In [10]:
import gensim.downloader as api
data = api.load("quora-duplicate-questions")
glove = api.load("glove-wiki-gigaword-100")

2020-02-10 21:42:23,353 : MainThread : INFO : loading projection weights from /Users/oliverborchers/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz
2020-02-10 21:43:05,757 : MainThread : INFO : loaded (400000, 100) matrix from /Users/oliverborchers/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz


In [11]:
sentences = []
for d in data:
    # Let's blow up the data a bit by replicating each sentence.
    for i in range(8):
        sentences.append(d["question1"].split())
        sentences.append(d["question2"].split())
s = IndexedList(sentences)
print(len(s))

/Users/oliverborchers/anaconda3/envs/fsedev/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


6468640


So we have about 6468640 sentences we want to compute the embeddings for. If you import the FAST_VERSION variable as follows you can ensure, that the compiliation of the cython routines worked correctly:

In [12]:
from fse.models.average import FAST_VERSION, MAX_WORDS_IN_BATCH
print(MAX_WORDS_IN_BATCH)
print(FAST_VERSION)
# 1 -> The fast version works

10000
1


In [13]:
from fse.models import uSIF
model = uSIF(glove, workers=2, lang_freq="en")

2020-02-10 21:43:26,025 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


In [14]:
model.train(s)

2020-02-10 21:43:26,444 : MainThread : INFO : scanning all indexed sentences and their word counts
2020-02-10 21:43:31,445 : MainThread : INFO : SCANNING : finished 4413745 sentences with 48816338 words
2020-02-10 21:43:33,734 : MainThread : WARNING : found 16 empty sentences
2020-02-10 21:43:33,735 : MainThread : INFO : finished scanning 6468640 sentences with an average length of 11 and 71556728 total words
2020-02-10 21:43:33,851 : MainThread : INFO : estimated memory for 6468640 sentences with 100 dimensions and 400000 vocabulary: 2621 MB (2 GB)
2020-02-10 21:43:33,852 : MainThread : INFO : initializing sentence vectors for 6468640 sentences
2020-02-10 21:43:56,067 : MainThread : INFO : pre-computing uSIF weights for 400000 words
2020-02-10 21:43:57,262 : MainThread : INFO : begin training
2020-02-10 21:44:02,273 : MainThread : INFO : PROGRESS : finished 37.80% with 2444873 sentences and 18587179 words, 488974 sentences/s
2020-02-10 21:44:07,274 : MainThread : INFO : PROGRESS : fin

(6468624, 49255184)

The models training speed revolves at around 400,000 - 500,000 sentences / seconds. That means we finish the task in about 10 seconds. This is **heavily dependent** on the input processing. If you train ram-to-ram it is naturally faster than any ram-to-disk or disk-to-disk varianty. Similarly, the speed depends on the workers.

Once the sif model is trained, you can perform additional inferences for unknown sentences. This two step process for new data is required, as computing the principal components for models like SIF and uSIF will require a fair amount of sentences. If you want the vector for a single sentence (which is out of the training vocab), just use:

In [15]:
tmp = ("Hello my friends".split(), 0)
model.infer([tmp])

2020-02-10 21:44:26,044 : MainThread : INFO : scanning all indexed sentences and their word counts
2020-02-10 21:44:26,045 : MainThread : INFO : finished scanning 1 sentences with an average length of 3 and 3 total words
2020-02-10 21:44:26,047 : MainThread : INFO : removing 5 principal components took 0s


array([[ 2.58718699e-01, -2.96060964e-02,  2.75402740e-02,
        -2.84367323e-01, -7.66123906e-02,  4.69245732e-01,
        -1.08420335e-01,  2.74900701e-02, -6.51940107e-02,
        -3.48900437e-01, -3.30639817e-03, -7.38123357e-02,
         1.99272603e-01,  1.58340886e-01, -1.19580366e-01,
        -2.94115573e-01,  9.44712311e-02, -1.60182863e-01,
        -3.77932310e-01,  3.62254620e-01, -1.04730584e-01,
         2.72801578e-01, -3.65233980e-02, -1.77455202e-01,
         1.13285437e-01,  9.37283933e-02, -2.23851919e-01,
        -5.82970530e-02,  4.76750970e-01,  1.19097173e-01,
         2.51136065e-01,  2.99976945e-01,  3.93524468e-01,
         1.26966879e-01,  1.19876862e-03,  2.52949506e-01,
         1.83217332e-01,  6.29579574e-02,  2.79819459e-01,
        -1.32508770e-01, -1.32991910e-01,  1.35885537e-01,
         2.27139533e-01, -1.15716822e-01, -1.42301470e-01,
        -1.17087245e-01, -4.09713805e-01,  3.27361971e-01,
         4.02728885e-01, -1.03995442e-01, -1.11777350e-0

## Querying the model

In order to query the model or perform similarity computations we can just access the model.sv (sentence vectors) object and use its method. To get a vector for an index, just call

In [16]:
model.sv[0]

array([ 0.06564295,  0.0012124 ,  0.02864488,  0.29741746,  0.16618916,
       -0.33291832, -0.25267577, -0.11754846, -0.00645616, -0.0986203 ,
       -0.03115754,  0.11605997, -0.06554113, -0.26570198, -0.19048208,
       -0.05398345, -0.00800271,  0.06935053,  0.02384207,  0.15339501,
        0.0931268 ,  0.04639681, -0.23096606,  0.1496515 , -0.14506361,
        0.02416093,  0.05317958,  0.06964332, -0.07533754,  0.006847  ,
       -0.2449986 ,  0.22674319, -0.09827837, -0.09429546,  0.13742915,
        0.15489256,  0.20663384, -0.10573711, -0.09373225, -0.21597916,
       -0.04622186, -0.07917423,  0.03237222, -0.09423919, -0.24972957,
        0.1362891 , -0.24006578,  0.05784579, -0.06796119, -0.49378857,
       -0.22550753, -0.00635221, -0.03531939,  0.2966177 , -0.17845063,
       -0.5473429 , -0.14862986, -0.03552294,  0.6726266 , -0.07657065,
        0.05169982, -0.18650085, -0.1508371 , -0.00102763,  0.05002424,
        0.14072034, -0.19600302,  0.21199626,  0.12934232, -0.07

To compute the similarity or distance between two sentence from the training set you can call:

In [17]:
print(model.sv.similarity(0,1).round(3))
print(model.sv.distance(0,1).round(3))

0.965
0.035


We can further call for the most similar sentences given an index. For example, we want to know the most similar sentences for sentence index 100:

In [18]:
print(s[100])

(['Should', 'I', 'buy', 'tiago?'], 100)


In [19]:
model.sv.most_similar(100)
# Division by zero can happen if you encounter empy sentences

2020-02-10 21:44:26,077 : MainThread : INFO : precomputing L2-norms of sentence vectors


[(3727921, 1.0),
 (1807119, 1.0),
 (3727935, 1.0),
 (3727933, 1.0),
 (3727931, 1.0),
 (3727929, 1.0),
 (3727927, 1.0),
 (3727925, 1.0),
 (3727923, 1.0),
 (599388, 1.0)]

However, the preceding function will only supply the indices of the most similar sentences. You can circumvent this problem by passing an indexable function to the most_similar call:

In [20]:
model.sv.most_similar(100, indexable=s.items)

[(['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727921, 1.0),
 (['Should', 'I', 'buy', 'Xiaomi', 'Redmi', 'Note', '3?', 'Why?'],
  1807119,
  1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727935, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727933, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727931, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727929, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727927, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727925, 1.0),
 (['Should', 'I', 'buy', 'KTM', 'Duke', '200?'], 3727923, 1.0),
 (['Should', 'I', 'buy', 'bitcoin?'], 599388, 1.0)]

There we go. This is a lot more understandable than the initial list of indices.

To search for sentences, which are similar to a given word vector, you can call:

In [21]:
model.sv.similar_by_word("easy", wv=glove, indexable=s.items)

[(['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717071,
  0.9463648796081543),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717059,
  0.9463648796081543),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717063,
  0.9463648796081543),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717067,
  0.9463648796081543),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717061,
  0.9463648796081543),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717065,
  0.9463648796081543),
 (['Which',
   'is',
   'mor

Furthermore, you can query for unknown (or new) sentences by calling:

In [22]:
model.sv.similar_by_sentence("Is this really easy to learn".split(), model=model, indexable=s.items)

2020-02-10 21:44:35,595 : MainThread : INFO : scanning all indexed sentences and their word counts
2020-02-10 21:44:35,597 : MainThread : INFO : finished scanning 1 sentences with an average length of 6 and 6 total words
2020-02-10 21:44:35,600 : MainThread : INFO : removing 5 principal components took 0s


[(['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255666,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255668,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418236,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418238,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255664,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418232,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255674,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255672,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418230,
  0.9860048294067383),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255670,
  0.9860048294067383)]

Feel free to browse through the library and get to know the functions a little better!